#### Show in table output gen

In [4]:
import os
import sys

parent_dir = os.path.abspath('..')
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

import pandas as pd
from constants import OW_TYPES, NORDIC_NODES

CASES = ["Base", "NOgrid", "NoHubs", "NoHubsNOgrid", "OnlyCentral", "OnlyEU", "OnlyNorth"]
DF_BY_CASES = dict({})

base_df = None

for case in CASES:
    _df = pd.read_csv(f'../Results/{case}/results_output_gen.csv')
    _df = _df[_df["Node"].isin(NORDIC_NODES)]
    _df["genInstalledCap_GW"] = _df["genInstalledCap_MW"] / 1000 # To GW
    _df = _df.drop(columns=["genInstalledCap_MW"])
    _df["genInstalledCap_GW"] = _df.groupby(['GeneratorType', 'Period'])['genInstalledCap_GW'].transform('sum')
    _df = _df.drop_duplicates(("GeneratorType", "Period"), ignore_index=True)[["GeneratorType", "Period", "genInstalledCap_GW"]]
    if case == "Base": 
        base_df = _df
    DF_BY_CASES[case] = _df

DF_BY_CASES

{'Base':     GeneratorType     Period  genInstalledCap_GW
 0    Coalexisting  2020-2025            5.963000
 1    Coalexisting  2025-2030            3.669538
 2    Coalexisting  2030-2035            2.465471
 3    Coalexisting  2035-2040            1.720096
 4    Coalexisting  2040-2045            1.192600
 ..            ...        ...                 ...
 171       Nuclear  2035-2040           11.752297
 172       Nuclear  2040-2045           18.209745
 173       Nuclear  2045-2050           23.402651
 174       Nuclear  2050-2055           21.899938
 175       Nuclear  2055-2060           21.899938
 
 [176 rows x 3 columns],
 'NOgrid':     GeneratorType     Period  genInstalledCap_GW
 0    Coalexisting  2020-2025            5.963000
 1    Coalexisting  2025-2030            3.669538
 2    Coalexisting  2030-2035            2.465471
 3    Coalexisting  2035-2040            1.720096
 4    Coalexisting  2040-2045            1.192600
 ..            ...        ...                 ...
 171 

Calculate diff to base

In [5]:
base_df = base_df.rename(columns={"genInstalledCap_GW": "base_genInstalledCap_GW"})
for case, _df in DF_BY_CASES.items():
    new_df = _df.merge(base_df, on=["GeneratorType", "Period"], how="left")
    new_df["Diff to base"] = round(new_df["genInstalledCap_GW"] - new_df["base_genInstalledCap_GW"])
    new_df["Diff to base %"] = round((new_df["genInstalledCap_GW"]/new_df["base_genInstalledCap_GW"] - 1) * 100, 1)
    new_df["genInstalledCap_GW"] = round(new_df["genInstalledCap_GW"])
    DF_BY_CASES[case] = new_df

Filter by case and period

In [7]:
FILTER_CASE = "Base"
FILTER_PERIOD = "2045-2050"

filtered_df = DF_BY_CASES[FILTER_CASE]
filtered_df = filtered_df[filtered_df["Period"] == FILTER_PERIOD].sort_values("genInstalledCap_GW", ascending=False).reset_index(drop=True)
print(f"Total offshore wind prod: {filtered_df[filtered_df["GeneratorType"].isin(OW_TYPES)]["genInstalledCap_GW"].sum()}GW")
filtered_df

Total offshore wind prod: 20.0GW


,GeneratorType,Period,genInstalledCap_GW,base_genInstalledCap_GW,Diff to base,Diff to base %
0,Windonshore,2045-2050,144.0,144.311832,0.0,0.0
1,Hydroregulated,2045-2050,46.0,45.856336,0.0,0.0
2,Hydrorun-of-the-river,2045-2050,26.0,25.592038,0.0,0.0
3,Nuclear,2045-2050,23.0,23.402651,0.0,0.0
4,Windoffshoregrounded,2045-2050,20.0,19.800004,0.0,0.0
5,Bio,2045-2050,19.0,18.530591,0.0,0.0
6,GasCCGT,2045-2050,12.0,11.547133,0.0,0.0
7,GasOCGT,2045-2050,11.0,11.084659,0.0,0.0
8,Waste,2045-2050,5.0,5.111000,0.0,0.0
9,Solar,2045-2050,2.0,1.567500,0.0,0.0
